In [1]:
from bs4 import BeautifulSoup
import requests
import json
import random
import time
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from lxml import html
import re



In [2]:
url =  'https://daily.afisha.ru/archive/volna/archive/pop/'
meat = requests.get(url).text

In [3]:
df_year=[]
df_artist=[]
df_song=[] 
soup = BeautifulSoup(meat, 'lxml')
cont=soup.body.find('div', class_='g-content').find('div', class_='page').find('article', class_='article article_common').find('ul').find_all('li')
for i in cont:
    try:
        df_artist.append(i.a.text.strip().split('\n')[0].strip().replace('«','').replace('»','').replace("'",""))
        df_song.append(i.a.text.strip().split('\n')[1].strip().replace('«','').replace('»','').replace("'",""))
        df_year.append('1000')
    except:
        pass

In [4]:
df_artist[0]+' '+df_song[0]

'Кар-Мэн Лондон, гуд-бай!'

In [5]:
df_lyrics_track=[]
df_lyrics=[]
df_genre=[]

for i in tqdm(range(0,len(df_artist))):
    song=df_song[i]
    meat = requests.get('https://www.google.com/search?q={}&ie=utf-8&oe=utf-8'.format(df_artist[i]+' '+df_song[i]+ 'текст megalyrics'))
    page=BeautifulSoup(meat.text)
    for i in page.body.find_all('div'):
        try:
            z = i.a['href'].split('/url?q=')[-1]
            if 'megalyrics.ru/lyric/' in z:
                break
        except:
            pass
    proxies = {'https': 'socks5h://198.27.75.152:1080', 'http': 'socks5h://198.27.75.152:1080'}
    meat = requests.get(z, verify=False, proxies=proxies).text
    page=BeautifulSoup(meat)
    d=page.find('div', class_="text_inner")
    d1=re.sub(r'<br/*?>', '\n', str(d))
    d1=BeautifulSoup(d1).text
    df_lyrics_track.append(song)
    df_lyrics.append(d1)
    df_genre.append('Русские треки')

C:\Users\Tom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [6]:
lyrics_df = pd.DataFrame({'track': df_lyrics_track,
'lyrics': df_lyrics,
'genre': df_genre
})
lyrics_df.tail()
 

,track,lyrics,genre
94,Режиссер,"Я закрою за собою, ты сказала ""я не стою""\nЯ п...",Русские треки
95,Любовь спасет мир,"Мир, в котором я живу, не делится на части \nП...",Русские треки
96,Прованс,Уютное кафе на улицах сплетённой мебелью\nГде ...,Русские треки
97,Больно,"Сколько можно искать тебя,\nСколько можно теря...",Русские треки
98,Чумачечая весна,Пришла и оторвала голову нам чумачечая весна\n...,Русские треки


In [7]:
tracks = pd.DataFrame({'year': df_year[0:99],
'artist': df_artist,
'track': df_song,
})
tracks.tail()

,year,artist,track
94,1000,Градусы,Режиссер
95,1000,Вера Брежнева,Любовь спасет мир
96,1000,Елка,Прованс
97,1000,Нюша,Больно
98,1000,Потап и Настя,Чумачечая весна


In [ ]:
from sqlalchemy import create_engine
engine = create_engine('KEY')
lyrics_df.loc[lyrics_df['lyrics']!='FAILURE'].to_sql('LYRICS', engine, if_exists='append')

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('KEY')
tracks.to_sql('TRACKS', engine, if_exists='append')d